In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier

In [4]:
df_dist = pd.read_csv('MS_dist.csv')
df_rank = pd.read_csv('TS_rolling_rankings.csv')
del df_dist['Unnamed: 0']
del df_rank['Unnamed: 0']

In [6]:
df_dist = df_dist[['WINNER','LOSER','Winner Distance','Loser Distance','match_week','Winner Home','Loser Home']]

In [10]:
df_rank.columns = ['WINNER', 'WINNER RATING', 'match_week']
df = pd.merge(df_dist,df_rank,on=['WINNER', 'match_week'], how = 'inner')
df_rank.columns = ['LOSER', 'LOSER RATING', 'match_week']
df = pd.merge(df,df_rank,on=['LOSER', 'match_week'], how = 'inner')

In [17]:
df.columns = ['Main Player','Opponent','MP Distance', 'Opponent Distance', 'match_week', 'MP home', 'Opponent Home', 'MP Rating', 'Opponent Rating']
df2 = df.copy(deep = 'True')
df2.columns =  ['Opponent','Main Player','Opponent Distance', 'MP Distance', 'match_week', 'Opponent Home', 'MP home', 'Opponent Rating', 'MP Rating']
df['win'] = [1]*len(df)
df2['win'] = [0]*len(df2)

df_main = df.append(df2).sort_values(by = 'match_week')

c:\users\andrew.zhuang\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [20]:
df_train = df_main[:-400]
df_test = df_main[400:]

In [33]:
features = ['MP Distance', 'Opponent Distance', 'match_week', 'MP home', 'Opponent Home', 'MP Rating', 'Opponent Rating']
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(df_train.dropna()[features], df_train.dropna()['win'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [34]:
pred = clf.predict(df_test.dropna()[features])
df_results = pd.DataFrame()
df_results['prediction'] = pred
df_results['actual'] = df_test.dropna()['win'].values
df_results['error'] = abs(df_results['prediction'] - df_results['actual'])

In [35]:
df_results['error'].mean()

0.29336188436830835

In [32]:
features = ['MP Rating', 'Opponent Rating']
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(df_train.dropna()[features], df_train.dropna()['win'])

pred = clf.predict(df_test.dropna()[features])
df_results = pd.DataFrame()
df_results['prediction'] = pred
df_results['actual'] = df_test.dropna()['win'].values
df_results['error'] = abs(df_results['prediction'] - df_results['actual'])

df_results['error'].mean()

0.25446109921484655

In [37]:
df_main[df_main['Main Player'] == 'Luis Ramon Garrido']

,MP Distance,MP Rating,MP home,Main Player,Opponent,Opponent Distance,Opponent Home,Opponent Rating,match_week,win
14,1737.735171,26.149177,0,Luis Ramon Garrido,Dewantoro Pandu,14542.771585,0,22.125312,20171219,0
17,1737.735171,26.149177,0,Luis Ramon Garrido,Sahil Sipani,18067.790158,0,11.843447,20171219,1
1465,2697.569158,26.868868,0,Luis Ramon Garrido,Pablo Sanchez,1552.128864,0,0.849920,20180517,1
1466,2697.569158,26.868868,0,Luis Ramon Garrido,Arturo Hernandez /,2697.569158,0,14.583958,20180517,1
1467,2697.569158,26.868868,0,Luis Ramon Garrido,Dino Delmastro,1552.128864,0,8.704490,20180517,1
1468,2697.569158,26.868868,0,Luis Ramon Garrido,Humblers Heymard /,1467.557903,0,11.587050,20180517,1
1469,2697.569158,26.868868,0,Luis Ramon Garrido,Jose Guevara /,126.036960,1,15.669320,20180517,1
